In [87]:
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import re # baray hazf adad
from collections import defaultdict

import math
import matplotlib.pyplot as plt

# skip_gram
import gensim
from gensim.models import Word2Vec

# GRU and RNN
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, GRU, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam


# padding
from keras.preprocessing.sequence import pad_sequences

In [2]:
unsupervised = np.load('unsupervised.npy', allow_pickle = True)
x_test = np.load('x_test.npy' , allow_pickle = True)
x_train = np.load('x_train.npy',allow_pickle = True)
x_val = np.load('x_val.npy', allow_pickle = True)
y_train = np.loadtxt('y_train.txt')
y_val = np.loadtxt('y_val.txt')

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# to sentense
sentense = []
for i in unsupervised:
  sentense.append(sent_tokenize(i))

In [5]:
# preprocess
stemmer = PorterStemmer()
words_unsupervised_sentence = []
for i in sentense:
    for j in i:
        tokens = word_tokenize(re.sub(r'\d+|[\n\r\t]|<br />', '', j))
        tokens = [word.lower() for word in tokens]
        translator = str.maketrans('','',punctuation)
        tokens = [word.translate(translator) for word in tokens]
        tokens = [word for word in tokens if not word in stop_words]
        tokens = [word for word in tokens if word != ""]
        tokens = [stemmer.stem(word) for word in tokens]
        words_unsupervised_sentence.append(tokens)

In [160]:
def find_counts(word_list):
    word_counts = defaultdict(int)
    for jomalat in word_list:
        for word in jomalat:
            word_counts[word] += 1
    return word_counts

def encode_sentence(sentence, vocab):
    keys = vocab.keys()
    return [word if word in keys else 'UNK' for word in sentence]

def separate_encodings(data, encoding_size):
    separated_encodings = []
    for sample in data:
        sample_encodings = []
        for encoding_dict in sample:
            sample_encodings.append(encoding_dict[encoding_size])
        separated_encodings.append(sample_encodings)
    return separated_encodings


def create_rnn_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(SimpleRNN(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# rnn with greater than layer size:
def create_rnn_model_gls(input_shape):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(128, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(SimpleRNN(128)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# add layer:
def create_rnn_model_al(input_shape):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(SimpleRNN(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(SimpleRNN(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(GRU(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# gru with greater than layer size:
def create_gru_model_gls(input_shape):
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(GRU(128)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# add layer
def create_gru_model_al(input_shape):
    model = Sequential()
    model.add(Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(GRU(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fine_tuned
def create_rnn_model_ft(input_shape, embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        input_length=input_shape,
                        weights=[embedding_matrix],
                        trainable=True))  # fine-tuning embedding
    model.add(Bidirectional(SimpleRNN(64, return_sequences=True)))
    model.add(Bidirectional(SimpleRNN(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# fine_tuned
def create_gru_model_ft(input_shape, embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        input_length=input_shape,
                        weights=[embedding_matrix],
                        trainable=True))  # fine-tuning embedding
    model.add(Bidirectional(GRU(64, return_sequences=True)))
    model.add(Bidirectional(GRU(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model


def text_to_sequence(text, word_to_index):
    return [word_to_index.get(word, word_to_index['UNK']) for word in text]

# def create_rnn_model_gpu(input_shape):
#   with tf.device('/device:GPU:0'):
#     model = Sequential()
#     model.add(Bidirectional(SimpleRNN(64, return_sequences=True), input_shape=input_shape))
#     model.add(Bidirectional(SimpleRNN(64)))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model

# def create_gru_model_gpu(input_shape):
#     with tf.device('/device:GPU:0'):
#       model = Sequential()
#       model.add(Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape))
#       model.add(Bidirectional(GRU(64)))
#       model.add(Dense(1, activation='sigmoid'))
#       model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [7]:
dictionary = find_counts(words_unsupervised_sentence)

In [8]:
sorted_dictionary = sorted(dictionary.items(), key=lambda item: item[1], reverse=True)

In [9]:
len(sorted_dictionary)

150349

In [10]:
top_10000_words = sorted_dictionary[:10000]
top_10000_dict = dict(top_10000_words)

In [11]:
top_10000_dict

{'movi': 100202,
 'film': 94673,
 'nt': 66799,
 'one': 53947,
 'like': 44553,
 'time': 30143,
 'make': 29502,
 'good': 29248,
 'charact': 28430,
 'get': 27968,
 'see': 27688,
 'watch': 26693,
 'would': 26634,
 'stori': 25015,
 'even': 24938,
 'realli': 23095,
 'scene': 21265,
 'look': 19587,
 'well': 19267,
 'much': 19093,
 'could': 18976,
 'show': 18717,
 'end': 18661,
 'bad': 18523,
 'peopl': 18508,
 'great': 18364,
 'go': 18355,
 'also': 17973,
 'love': 17540,
 'first': 17464,
 'play': 17332,
 'think': 17183,
 'way': 17175,
 'act': 16838,
 'thing': 16250,
 'made': 15593,
 'know': 14807,
 'seem': 14448,
 'say': 14285,
 'plot': 13585,
 'two': 13453,
 'mani': 13267,
 'take': 13266,
 'want': 13203,
 'come': 13186,
 'work': 13169,
 'never': 12936,
 'actor': 12925,
 'tri': 12909,
 'seen': 12772,
 'littl': 12736,
 'year': 12452,
 'best': 12384,
 'life': 11818,
 'give': 11642,
 'ever': 11636,
 'better': 11527,
 'find': 11064,
 'man': 11016,
 'perform': 10838,
 'still': 10576,
 'use': 10426,

In [12]:
remaining_words = sorted_dictionary[10000:]

In [13]:
top_10000_dict['UNK'] = sum(count for word, count in remaining_words)

In [14]:
encoded_sentences = [encode_sentence(sentence, top_10000_dict) for sentence in words_unsupervised_sentence]

In [15]:
# skip_gram

vector_sizes = [32, 64, 128]
window_size = 5
min_count = 1
sg = 1 # sg = 1 -> skip_gram sg = 0 -> CBOW

models = {}
for size in vector_sizes:
    model = Word2Vec(encoded_sentences, vector_size=size, window=window_size, min_count=min_count, sg=sg)
    models[size] = model

In [178]:
# simple test:
word = 'movi'
for size in vector_sizes:
    if word in models[size].wv:
        print(f"Vector for '{word}' with size {size}:")
        print(models[size].wv[word])
    else:
        print(f"'{word}' not in vocabulary for size {size}")
        print(models[size].wv['UNK'])

Vector for 'movi' with size 32:
[-0.55242807 -0.10878559  0.61112684  0.2557324  -0.01671135 -0.3611456
 -0.04515338  0.18593906  0.1267472   0.16044836  0.20868947 -0.42697015
  0.04891961 -0.19592138 -0.22675757  0.12587309 -0.21449465  0.5338716
  0.353872   -0.0302577   0.1204041   0.52173597  0.79954934  0.12733048
 -0.03968844  0.14120534  0.05270621  0.13879393  0.08903056 -0.36504588
 -0.44362235  0.1276386 ]
Vector for 'movi' with size 64:
[ 0.05426544 -0.02013987  0.41944084 -0.18796434 -0.36839667 -0.240543
  0.23282738  0.11941769  0.05330034  0.06443696  0.5208424  -0.01314897
  0.47019064 -0.08743519 -0.45606402  0.2963877  -0.08796734  0.09610136
 -0.13870391  0.02642799  0.05426032  0.21557021  0.4408834  -0.4669063
  0.03171263  0.3257347  -0.4015015   0.33352754  0.40719625 -0.16908988
  0.24119061  0.02248289 -0.32193398 -0.12685758 -0.01371621 -0.08278523
  0.34808975 -0.23704278  0.08373534  0.13351835 -0.0263369   0.2700327
  0.10416241 -0.0861238   0.11093234  0.

In [17]:
x_train[0]

'One of the best movies for all ages. You will never be able to look at LION KING again without thinking of the extra history this movie adds. Nearly 40 years old, I watched this with my wife and two sons after work tonight & I have not laughed & enjoyed a movie so much in a long time.<br /><br />Take time out and watch this with the kids. It will remind you of how Disney used to be when you were a young one.'

In [18]:
# preprocess x_train
stemmer = PorterStemmer()
x_trian_preprocessed = []
for i in x_train:
    tokens = word_tokenize(re.sub(r'\d+|[\n\r\t]|<br />', '', i))
    tokens = [word.lower() for word in tokens]
    translator = str.maketrans('','',punctuation)
    tokens = [word.translate(translator) for word in tokens]
    tokens = [word for word in tokens if not word in stop_words]
    tokens = [word for word in tokens if word != ""]
    tokens = [stemmer.stem(word) for word in tokens]
    x_trian_preprocessed.append(tokens)

In [19]:
# encoding x_train:
x_trian_encoded = [encode_sentence(sentence, top_10000_dict) for sentence in x_trian_preprocessed]


In [20]:
x_train_sg_encoded = []
for words in x_trian_encoded:
    words_sg_encoded = []
    for word in words[:150]:
        word_sg_encoded = {}
        for size in vector_sizes:
            word_sg_encoded[size] = []
        for size in vector_sizes:
            if word in models[size].wv:
                word_sg_encoded[size] = models[size].wv[word]
            else:
                word_sg_encoded[size] = models[size].wv['UNK']
        words_sg_encoded.append(word_sg_encoded)
    x_train_sg_encoded.append(words_sg_encoded)

In [21]:
# check kardan inke dorost encode kardam ya na :
x_train_sg_encoded[1][0]

{32: array([ 0.08589874,  0.00633425,  0.8326719 ,  0.50904477,  0.20808986,
        -0.6930948 ,  0.3884629 ,  0.12366667, -0.37081218,  0.64167523,
         0.15540002, -0.5570943 , -0.3352496 , -0.20650181, -0.2053635 ,
         0.02259283, -0.08307917, -0.2687757 ,  0.5046663 , -0.01766053,
         0.04076756,  0.61723375, -0.08698864,  0.4199523 ,  0.41424003,
        -0.2039989 , -0.17195262, -0.17048974,  0.26476982,  0.02114576,
        -0.03123811,  0.1805645 ], dtype=float32),
 64: array([ 2.6992494e-01,  1.3049598e-01,  9.2131764e-02, -1.8935843e-01,
         2.7332302e-02, -3.2378420e-01, -7.3939994e-02, -3.1959763e-01,
        -5.0565374e-01,  7.3656626e-02,  4.8746858e-02,  1.3634032e-01,
        -2.8222647e-01,  2.8042197e-01, -2.5212482e-01,  3.2286966e-01,
        -1.0571757e-01,  6.4682752e-02, -2.0214927e-01,  3.6099759e-01,
         3.9949822e-01,  7.4453945e-03,  3.9921215e-01, -9.9847817e-01,
        -1.3148864e-01,  2.2727890e-01, -1.8164442e-01,  1.1620398e-01,

In [22]:
# check kardan inke dorost encode kardam ya na :
word = x_trian_preprocessed[1][0]
for size in vector_sizes:
    if word in models[size].wv:
        print(f"Vector for '{word}' with size {size}:")
        print(models[size].wv[word])
    else:
        print(f"'{word}' not in vocabulary for size {size}")
        print(models[size].wv['UNK'])

Vector for 'explain' with size 32:
[ 0.08589874  0.00633425  0.8326719   0.50904477  0.20808986 -0.6930948
  0.3884629   0.12366667 -0.37081218  0.64167523  0.15540002 -0.5570943
 -0.3352496  -0.20650181 -0.2053635   0.02259283 -0.08307917 -0.2687757
  0.5046663  -0.01766053  0.04076756  0.61723375 -0.08698864  0.4199523
  0.41424003 -0.2039989  -0.17195262 -0.17048974  0.26476982  0.02114576
 -0.03123811  0.1805645 ]
Vector for 'explain' with size 64:
[ 2.6992494e-01  1.3049598e-01  9.2131764e-02 -1.8935843e-01
  2.7332302e-02 -3.2378420e-01 -7.3939994e-02 -3.1959763e-01
 -5.0565374e-01  7.3656626e-02  4.8746858e-02  1.3634032e-01
 -2.8222647e-01  2.8042197e-01 -2.5212482e-01  3.2286966e-01
 -1.0571757e-01  6.4682752e-02 -2.0214927e-01  3.6099759e-01
  3.9949822e-01  7.4453945e-03  3.9921215e-01 -9.9847817e-01
 -1.3148864e-01  2.2727890e-01 -1.8164442e-01  1.1620398e-01
 -1.7947803e-01 -4.5531434e-01 -1.7840551e-01  2.1242754e-01
  4.2428277e-02 -2.4147815e-01 -2.2453319e-01 -1.953050

In [24]:
encodings_32 = separate_encodings(x_train_sg_encoded, 32)
encodings_64 = separate_encodings(x_train_sg_encoded, 64)
encodings_128 = separate_encodings(x_train_sg_encoded, 128)

In [25]:
max_length = 150
encodings_32_padded = pad_sequences(encodings_32, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_64_padded = pad_sequences(encodings_64, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_128_padded = pad_sequences(encodings_128, maxlen=max_length, dtype='float32', padding='post', truncating='post')

In [43]:
# preprocess x_val
stemmer = PorterStemmer()
x_val_preprocessed = []
for i in x_val:
    tokens = word_tokenize(re.sub(r'\d+|[\n\r\t]|<br />', '', i))
    tokens = [word.lower() for word in tokens]
    translator = str.maketrans('','',punctuation)
    tokens = [word.translate(translator) for word in tokens]
    tokens = [word for word in tokens if not word in stop_words]
    tokens = [word for word in tokens if word != ""]
    tokens = [stemmer.stem(word) for word in tokens]
    x_val_preprocessed.append(tokens)
# encoding x_val:
x_val_encoded = [encode_sentence(sentence, top_10000_dict) for sentence in x_val_preprocessed]

x_val_sg_encoded = []
for words in x_val_encoded:
    words_val_sg_encoded = []
    for word in words[:150]:
        word_val_sg_encoded = {}
        for size in vector_sizes:
            word_val_sg_encoded[size] = []
        for size in vector_sizes:
            if word in models[size].wv:
                word_val_sg_encoded[size] = models[size].wv[word]
            else:
                word_val_sg_encoded[size] = models[size].wv['UNK']
        words_val_sg_encoded.append(word_val_sg_encoded)
    x_val_sg_encoded.append(words_val_sg_encoded)
    
encodings_val_32 = separate_encodings(x_val_sg_encoded, 32)
encodings_val_64 = separate_encodings(x_val_sg_encoded, 64)
encodings_val_128 = separate_encodings(x_val_sg_encoded, 128)

encodings_val_32_padded = pad_sequences(encodings_val_32, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_val_64_padded = pad_sequences(encodings_val_64, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_val_128_padded = pad_sequences(encodings_val_128, maxlen=max_length, dtype='float32', padding='post', truncating='post')

In [31]:
rnn_model_32 = create_rnn_model((max_length, 32))
rnn_model_32.fit(encodings_32_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 61s 74ms/step - loss: 0.6160 - accuracy: 0.6678
Epoch 2/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6130 - accuracy: 0.6663
Epoch 3/10
782/782 [==============================] - 58s 74ms/step - loss: 0.5902 - accuracy: 0.6915
Epoch 4/10
782/782 [==============================] - 57s 74ms/step - loss: 0.6089 - accuracy: 0.6658
Epoch 5/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6422 - accuracy: 0.6255
Epoch 6/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6132 - accuracy: 0.6625
Epoch 7/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6077 - accuracy: 0.6663
Epoch 8/10
782/782 [==============================] - 57s 73ms/step - loss: 0.5864 - accuracy: 0.6960
Epoch 9/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6187 - accuracy: 0.6546
Epoch 10/10
782/782 [==============================] - 58s 75ms/step - loss: 0.622

In [44]:
rnn_eval_32 =  rnn_model_32.evaluate(encodings_val_32_padded, y_val)
print(f'RNN Validation Accuracy: {rnn_eval_32[1] * 100:.2f}%')

391/391 [==============================] - 9s 22ms/step - loss: 0.6416 - accuracy: 0.6315
RNN Validation Accuracy: 63.15%


In [35]:
gru_model_32 = create_gru_model((max_length, 32))
gru_model_32.fit(encodings_32_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 97s 115ms/step - loss: 0.4853 - accuracy: 0.7630
Epoch 2/10
782/782 [==============================] - 90s 116ms/step - loss: 0.3596 - accuracy: 0.8421
Epoch 3/10
782/782 [==============================] - 89s 114ms/step - loss: 0.3338 - accuracy: 0.8546
Epoch 4/10
782/782 [==============================] - 90s 115ms/step - loss: 0.3206 - accuracy: 0.8617
Epoch 5/10
782/782 [==============================] - 89s 114ms/step - loss: 0.3099 - accuracy: 0.8663
Epoch 6/10
782/782 [==============================] - 89s 114ms/step - loss: 0.2962 - accuracy: 0.8731
Epoch 7/10
782/782 [==============================] - 89s 114ms/step - loss: 0.2839 - accuracy: 0.8797
Epoch 8/10
782/782 [==============================] - 91s 116ms/step - loss: 0.2677 - accuracy: 0.8865
Epoch 9/10
782/782 [==============================] - 89s 114ms/step - loss: 0.2513 - accuracy: 0.8962
Epoch 10/10
782/782 [==============================] - 89s 114ms/step - l

In [45]:
gru_eval_32 =  gru_model_32.evaluate(encodings_val_32_padded, y_val)
print(f'GRU Validation Accuracy: {gru_eval_32[1] * 100:.2f}%')

391/391 [==============================] - 15s 39ms/step - loss: 0.3303 - accuracy: 0.8694
GRU Validation Accuracy: 86.94%


In [46]:
rnn_model_64 = create_rnn_model((max_length, 64))
rnn_model_64.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 60s 72ms/step - loss: 0.6326 - accuracy: 0.6434
Epoch 2/10
782/782 [==============================] - 56s 72ms/step - loss: 0.6096 - accuracy: 0.6713
Epoch 3/10
782/782 [==============================] - 57s 72ms/step - loss: 0.6274 - accuracy: 0.6506
Epoch 4/10
782/782 [==============================] - 57s 73ms/step - loss: 0.6201 - accuracy: 0.6572
Epoch 5/10
782/782 [==============================] - 57s 73ms/step - loss: 0.5985 - accuracy: 0.6778
Epoch 6/10
782/782 [==============================] - 57s 73ms/step - loss: 0.5933 - accuracy: 0.6834
Epoch 7/10
782/782 [==============================] - 57s 73ms/step - loss: 0.5947 - accuracy: 0.6786
Epoch 8/10
782/782 [==============================] - 57s 72ms/step - loss: 0.5894 - accuracy: 0.6880
Epoch 9/10
782/782 [==============================] - 58s 74ms/step - loss: 0.5555 - accuracy: 0.7192
Epoch 10/10
782/782 [==============================] - 58s 74ms/step - loss: 0.604

In [159]:
rnn_eval_64 =  rnn_model_64.evaluate(encodings_val_64_padded, y_val)
print(f'RNN Validation Accuracy: {rnn_eval_64[1] * 100:.2f}%')

391/391 [==============================] - 9s 22ms/step - loss: 0.5639 - accuracy: 0.7248
RNN Validation Accuracy: 72.48%


In [49]:
gru_model_64 = create_gru_model((max_length, 64))
gru_model_64.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 98s 116ms/step - loss: 0.4340 - accuracy: 0.7992
Epoch 2/10
782/782 [==============================] - 90s 116ms/step - loss: 0.3275 - accuracy: 0.8577
Epoch 3/10
782/782 [==============================] - 91s 116ms/step - loss: 0.3068 - accuracy: 0.8690
Epoch 4/10
782/782 [==============================] - 90s 116ms/step - loss: 0.2900 - accuracy: 0.8758
Epoch 5/10
782/782 [==============================] - 91s 116ms/step - loss: 0.2740 - accuracy: 0.8835
Epoch 6/10
782/782 [==============================] - 91s 116ms/step - loss: 0.2585 - accuracy: 0.8924
Epoch 7/10
782/782 [==============================] - 90s 115ms/step - loss: 0.2403 - accuracy: 0.9002
Epoch 8/10
782/782 [==============================] - 90s 115ms/step - loss: 0.2178 - accuracy: 0.9119
Epoch 9/10
782/782 [==============================] - 90s 116ms/step - loss: 0.1907 - accuracy: 0.9241
Epoch 10/10
782/782 [==============================] - 91s 117ms/step - l

In [50]:
gru_eval_64 =  gru_model_64.evaluate(encodings_val_64_padded, y_val)
print(f'RNN Validation Accuracy: {gru_eval_64[1] * 100:.2f}%')

391/391 [==============================] - 16s 37ms/step - loss: 0.3490 - accuracy: 0.8619
RNN Validation Accuracy: 86.19%


In [51]:
rnn_model_128 = create_rnn_model((max_length, 128))
rnn_model_128.fit(encodings_128_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 62s 75ms/step - loss: 0.6316 - accuracy: 0.6452
Epoch 2/10
782/782 [==============================] - 57s 73ms/step - loss: 0.5714 - accuracy: 0.7091
Epoch 3/10
782/782 [==============================] - 58s 74ms/step - loss: 0.5598 - accuracy: 0.7145
Epoch 4/10
782/782 [==============================] - 59s 75ms/step - loss: 0.6199 - accuracy: 0.6550
Epoch 5/10
782/782 [==============================] - 58s 74ms/step - loss: 0.6267 - accuracy: 0.6478
Epoch 6/10
782/782 [==============================] - 58s 74ms/step - loss: 0.6101 - accuracy: 0.6668
Epoch 7/10
782/782 [==============================] - 58s 74ms/step - loss: 0.6355 - accuracy: 0.6354
Epoch 8/10
782/782 [==============================] - 58s 74ms/step - loss: 0.6171 - accuracy: 0.6620
Epoch 9/10
782/782 [==============================] - 58s 74ms/step - loss: 0.6379 - accuracy: 0.6352
Epoch 10/10
782/782 [==============================] - 58s 74ms/step - loss: 0.603

In [53]:
rnn_eval_128 =  rnn_model_128.evaluate(encodings_val_128_padded, y_val)
print(f'RNN Validation Accuracy: {rnn_eval_128[1] * 100:.2f}%')

391/391 [==============================] - 9s 21ms/step - loss: 0.7033 - accuracy: 0.5466
RNN Validation Accuracy: 54.66%


In [52]:
gru_model_128 = create_gru_model((max_length, 128))
gru_model_128.fit(encodings_128_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 97s 114ms/step - loss: 0.4235 - accuracy: 0.8032
Epoch 2/10
782/782 [==============================] - 90s 115ms/step - loss: 0.3158 - accuracy: 0.8644
Epoch 3/10
782/782 [==============================] - 90s 114ms/step - loss: 0.2940 - accuracy: 0.8764
Epoch 4/10
782/782 [==============================] - 89s 114ms/step - loss: 0.2712 - accuracy: 0.8884
Epoch 5/10
782/782 [==============================] - 90s 114ms/step - loss: 0.2534 - accuracy: 0.8965
Epoch 6/10
782/782 [==============================] - 90s 115ms/step - loss: 0.2330 - accuracy: 0.9056
Epoch 7/10
782/782 [==============================] - 89s 114ms/step - loss: 0.2073 - accuracy: 0.9174
Epoch 8/10
782/782 [==============================] - 89s 113ms/step - loss: 0.1813 - accuracy: 0.9294
Epoch 9/10
782/782 [==============================] - 90s 116ms/step - loss: 0.1462 - accuracy: 0.9454
Epoch 10/10
782/782 [==============================] - 89s 114ms/step - l

In [56]:
gru_eval_128 =  gru_model_128.evaluate(encodings_val_128_padded, y_val)
print(f'RNN Validation Accuracy: {gru_eval_128[1] * 100:.2f}%')

391/391 [==============================] - 16s 38ms/step - loss: 0.4146 - accuracy: 0.8644
RNN Validation Accuracy: 86.44%


In [149]:
# try1 -> size = 32
embedding_matrix_try1 = []
for i in top_10000_dict.keys():
    embedding_matrix_try1.append(models[32].wv[i])

In [122]:
# try2 -> size = 64
embedding_matrix_try2 = []
for i in top_10000_dict.keys():
    embedding_matrix_try2.append(models[64].wv[i])

In [154]:
# try3 -> size = 128
embedding_matrix_try3 = []
for i in top_10000_dict.keys():
    embedding_matrix_try3.append(models[128].wv[i])

In [124]:
word_to_index = {word: index for index, word in dict(enumerate(top_10000_dict)).items()}

In [125]:
word_to_index

{'movi': 0,
 'film': 1,
 'nt': 2,
 'one': 3,
 'like': 4,
 'time': 5,
 'make': 6,
 'good': 7,
 'charact': 8,
 'get': 9,
 'see': 10,
 'watch': 11,
 'would': 12,
 'stori': 13,
 'even': 14,
 'realli': 15,
 'scene': 16,
 'look': 17,
 'well': 18,
 'much': 19,
 'could': 20,
 'show': 21,
 'end': 22,
 'bad': 23,
 'peopl': 24,
 'great': 25,
 'go': 26,
 'also': 27,
 'love': 28,
 'first': 29,
 'play': 30,
 'think': 31,
 'way': 32,
 'act': 33,
 'thing': 34,
 'made': 35,
 'know': 36,
 'seem': 37,
 'say': 38,
 'plot': 39,
 'two': 40,
 'mani': 41,
 'take': 42,
 'want': 43,
 'come': 44,
 'work': 45,
 'never': 46,
 'actor': 47,
 'tri': 48,
 'seen': 49,
 'littl': 50,
 'year': 51,
 'best': 52,
 'life': 53,
 'give': 54,
 'ever': 55,
 'better': 56,
 'find': 57,
 'man': 58,
 'perform': 59,
 'still': 60,
 'use': 61,
 'part': 62,
 'feel': 63,
 'someth': 64,
 'actual': 65,
 'interest': 66,
 'lot': 67,
 'back': 68,
 'real': 69,
 'director': 70,
 'guy': 71,
 'live': 72,
 'cast': 73,
 'though': 74,
 'new': 75,
 'f

In [131]:
sequences = [text_to_sequence(text, word_to_index) for text in x_trian_encoded]

In [132]:
x_train_ft = pad_sequences(sequences, maxlen=150 , padding = 'post', value = 10000)

In [141]:
sequences_val = [text_to_sequence(text, word_to_index) for text in x_val_encoded]

In [142]:
x_val_ft = pad_sequences(sequences_val, maxlen=150 , padding = 'post', value = 10000)

In [150]:
rnn_model_32_ft = create_rnn_model_ft(max_length,np.array(embedding_matrix_try1))
rnn_model_32_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 65s 79ms/step - loss: 0.6255 - accuracy: 0.6532
Epoch 2/10
782/782 [==============================] - 61s 78ms/step - loss: 0.5126 - accuracy: 0.7552
Epoch 3/10
782/782 [==============================] - 61s 79ms/step - loss: 0.4734 - accuracy: 0.7838
Epoch 4/10
782/782 [==============================] - 61s 78ms/step - loss: 0.4187 - accuracy: 0.8142
Epoch 5/10
782/782 [==============================] - 61s 79ms/step - loss: 0.5376 - accuracy: 0.7276
Epoch 6/10
782/782 [==============================] - 61s 78ms/step - loss: 0.5577 - accuracy: 0.7151
Epoch 7/10
782/782 [==============================] - 62s 79ms/step - loss: 0.5124 - accuracy: 0.7485
Epoch 8/10
782/782 [==============================] - 61s 79ms/step - loss: 0.4395 - accuracy: 0.8002
Epoch 9/10
782/782 [==============================] - 61s 78ms/step - loss: 0.4714 - accuracy: 0.7744
Epoch 10/10
782/782 [==============================] - 62s 80ms/step - loss: 0.425

In [151]:
rnn_eval_32_ft =  rnn_model_32_ft.evaluate(x_val_ft, y_val)
print(f'RNN Validation (fine_tuned) Accuracy: {rnn_eval_32_ft[1] * 100:.2f}%')

391/391 [==============================] - 9s 21ms/step - loss: 0.4714 - accuracy: 0.8060
RNN Validation (fine_tuned) Accuracy: 80.60%


In [152]:
gru_model_32_ft = create_gru_model_ft(max_length,np.array(embedding_matrix_try1))
gru_model_32_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 99s 117ms/step - loss: 0.4195 - accuracy: 0.8025
Epoch 2/10
782/782 [==============================] - 91s 117ms/step - loss: 0.2562 - accuracy: 0.8951
Epoch 3/10
782/782 [==============================] - 92s 117ms/step - loss: 0.2029 - accuracy: 0.9202
Epoch 4/10
782/782 [==============================] - 92s 117ms/step - loss: 0.1525 - accuracy: 0.9461
Epoch 5/10
782/782 [==============================] - 91s 117ms/step - loss: 0.1046 - accuracy: 0.9642
Epoch 6/10
782/782 [==============================] - 94s 120ms/step - loss: 0.0691 - accuracy: 0.9789
Epoch 7/10
782/782 [==============================] - 92s 118ms/step - loss: 0.0482 - accuracy: 0.9856
Epoch 8/10
782/782 [==============================] - 91s 116ms/step - loss: 0.0352 - accuracy: 0.9890
Epoch 9/10
782/782 [==============================] - 93s 119ms/step - loss: 0.0256 - accuracy: 0.9927
Epoch 10/10
782/782 [==============================] - 92s 118ms/step - l

In [155]:
gru_eval_32_ft =  gru_model_32_ft.evaluate(x_val_ft, y_val)
print(f'GRU Validation (fine_tuned) Accuracy: {gru_eval_32_ft[1] * 100:.2f}%')

391/391 [==============================] - 14s 36ms/step - loss: 0.6800 - accuracy: 0.8622
GRU Validation (fine_tuned) Accuracy: 86.22%


In [140]:
rnn_model_64_ft = create_rnn_model_ft(max_length,np.array(embedding_matrix_try2))
rnn_model_64_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 69s 85ms/step - loss: 0.5790 - accuracy: 0.6920
Epoch 2/10
782/782 [==============================] - 66s 84ms/step - loss: 0.5301 - accuracy: 0.7341
Epoch 3/10
782/782 [==============================] - 66s 84ms/step - loss: 0.5350 - accuracy: 0.7293
Epoch 4/10
782/782 [==============================] - 66s 84ms/step - loss: 0.4604 - accuracy: 0.7876
Epoch 5/10
782/782 [==============================] - 66s 84ms/step - loss: 0.4399 - accuracy: 0.8008
Epoch 6/10
782/782 [==============================] - 66s 84ms/step - loss: 0.3441 - accuracy: 0.8550
Epoch 7/10
782/782 [==============================] - 66s 84ms/step - loss: 0.3302 - accuracy: 0.8614
Epoch 8/10
782/782 [==============================] - 66s 84ms/step - loss: 0.2982 - accuracy: 0.8790
Epoch 9/10
782/782 [==============================] - 66s 84ms/step - loss: 0.3319 - accuracy: 0.8586
Epoch 10/10
782/782 [==============================] - 66s 85ms/step - loss: 0.245

In [145]:
rnn_eval_64_ft =  rnn_model_64_ft.evaluate(x_val_ft, y_val)
print(f'RNN Validation (fine_tuned) Accuracy: {rnn_eval_64_ft[1] * 100:.2f}%')

391/391 [==============================] - 9s 22ms/step - loss: 0.4991 - accuracy: 0.7787
RNN Validation (fine_tuned) Accuracy: 77.87%


In [147]:
gru_model_64_ft = create_gru_model_ft(max_length,np.array(embedding_matrix_try2))
gru_model_64_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 101s 120ms/step - loss: 0.4009 - accuracy: 0.8167
Epoch 2/10
782/782 [==============================] - 94s 120ms/step - loss: 0.2502 - accuracy: 0.9011
Epoch 3/10
782/782 [==============================] - 94s 120ms/step - loss: 0.1892 - accuracy: 0.9275
Epoch 4/10
782/782 [==============================] - 94s 120ms/step - loss: 0.1381 - accuracy: 0.9507
Epoch 5/10
782/782 [==============================] - 94s 121ms/step - loss: 0.0908 - accuracy: 0.9684
Epoch 6/10
782/782 [==============================] - 94s 120ms/step - loss: 0.0571 - accuracy: 0.9811
Epoch 7/10
782/782 [==============================] - 93s 119ms/step - loss: 0.0338 - accuracy: 0.9904
Epoch 8/10
782/782 [==============================] - 94s 120ms/step - loss: 0.0240 - accuracy: 0.9931
Epoch 9/10
782/782 [==============================] - 94s 120ms/step - loss: 0.0162 - accuracy: 0.9955
Epoch 10/10
782/782 [==============================] - 94s 120ms/step - 

In [148]:
gru_eval_64_ft =  gru_model_64_ft.evaluate(x_val_ft, y_val)
print(f'GRU Validation (fine_tuned) Accuracy: {gru_eval_64_ft[1] * 100:.2f}%')

391/391 [==============================] - 16s 36ms/step - loss: 0.7293 - accuracy: 0.8639
GRU Validation (fine_tuned) Accuracy: 86.39%


In [ ]:
rnn_model_128_ft = create_rnn_model_ft(max_length,np.array(embedding_matrix_try3))
rnn_model_128_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

In [ ]:
rnn_eval_128_ft =  rnn_model_128_ft.evaluate(x_val_ft, y_val)
print(f'RNN Validation (fine_tuned) Accuracy: {rnn_eval_128_ft[1] * 100:.2f}%')

In [ ]:
gru_model_128_ft = create_gru_model_ft(max_length,np.array(embedding_matrix_try3))
gru_model_128_ft.fit(x_train_ft, y_train, epochs=10, batch_size=32)

In [ ]:
gru_eval_128_ft =  gru_model_128_ft.evaluate(x_val_ft, y_val)
print(f'GRU Validation (fine_tuned) Accuracy: {gru_eval_128_ft[1] * 100:.2f}%')

In [157]:
rnn_model_64_gls = create_rnn_model_gls((max_length, 64))
rnn_model_64_gls.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 76s 93ms/step - loss: 0.6540 - accuracy: 0.6174
Epoch 2/10
782/782 [==============================] - 73s 94ms/step - loss: 0.6406 - accuracy: 0.6331
Epoch 3/10
782/782 [==============================] - 75s 96ms/step - loss: 0.6548 - accuracy: 0.6085
Epoch 4/10
782/782 [==============================] - 76s 97ms/step - loss: 0.6311 - accuracy: 0.6343
Epoch 5/10
782/782 [==============================] - 75s 96ms/step - loss: 0.6314 - accuracy: 0.6408
Epoch 6/10
782/782 [==============================] - 74s 95ms/step - loss: 0.6420 - accuracy: 0.6267
Epoch 7/10
782/782 [==============================] - 74s 94ms/step - loss: 0.6225 - accuracy: 0.6513
Epoch 8/10
782/782 [==============================] - 74s 95ms/step - loss: 0.6272 - accuracy: 0.6451
Epoch 9/10
782/782 [==============================] - 74s 94ms/step - loss: 0.6219 - accuracy: 0.6436
Epoch 10/10
782/782 [==============================] - 73s 94ms/step - loss: 0.612

In [158]:
rnn_eval_64_gls =  rnn_model_64_gls.evaluate(encodings_val_64_padded, y_val)
print(f'RNN Validation Accuracy: {rnn_eval_64_gls[1] * 100:.2f}%')

391/391 [==============================] - 18s 44ms/step - loss: 0.6228 - accuracy: 0.6493
RNN Validation Accuracy: 64.93%


In [161]:
gru_model_64_gls = create_gru_model_gls((max_length, 64))
gru_model_64_gls.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 257s 306ms/step - loss: 0.4312 - accuracy: 0.8009
Epoch 2/10
782/782 [==============================] - 236s 302ms/step - loss: 0.3253 - accuracy: 0.8606
Epoch 3/10
782/782 [==============================] - 238s 304ms/step - loss: 0.3033 - accuracy: 0.8702
Epoch 4/10
782/782 [==============================] - 250s 320ms/step - loss: 0.2842 - accuracy: 0.8797
Epoch 5/10
782/782 [==============================] - 242s 309ms/step - loss: 0.2656 - accuracy: 0.8875
Epoch 6/10
782/782 [==============================] - 241s 308ms/step - loss: 0.2429 - accuracy: 0.9000
Epoch 7/10
782/782 [==============================] - 238s 304ms/step - loss: 0.2122 - accuracy: 0.9138
Epoch 8/10
782/782 [==============================] - 239s 306ms/step - loss: 0.1741 - accuracy: 0.9312
Epoch 9/10
782/782 [==============================] - 239s 305ms/step - loss: 0.1300 - accuracy: 0.9508
Epoch 10/10
782/782 [==============================] - 240s 306m

In [162]:
gru_eval_64_gls =  gru_model_64_gls.evaluate(encodings_val_64_padded, y_val)
print(f'RNN Validation Accuracy: {gru_eval_64_gls[1] * 100:.2f}%')

391/391 [==============================] - 30s 72ms/step - loss: 0.4486 - accuracy: 0.8634
RNN Validation Accuracy: 86.34%


In [163]:
rnn_model_64_al = create_rnn_model_al((max_length, 64))
rnn_model_64_al.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 91s 110ms/step - loss: 0.6202 - accuracy: 0.6542
Epoch 2/10
782/782 [==============================] - 87s 112ms/step - loss: 0.6276 - accuracy: 0.6411
Epoch 3/10
782/782 [==============================] - 86s 110ms/step - loss: 0.6329 - accuracy: 0.6284
Epoch 4/10
782/782 [==============================] - 87s 112ms/step - loss: 0.6084 - accuracy: 0.6592
Epoch 5/10
782/782 [==============================] - 86s 111ms/step - loss: 0.6134 - accuracy: 0.6568
Epoch 6/10
782/782 [==============================] - 86s 110ms/step - loss: 0.6211 - accuracy: 0.6432
Epoch 7/10
782/782 [==============================] - 85s 108ms/step - loss: 0.6250 - accuracy: 0.6450
Epoch 8/10
782/782 [==============================] - 85s 108ms/step - loss: 0.6115 - accuracy: 0.6585
Epoch 9/10
782/782 [==============================] - 85s 108ms/step - loss: 0.6232 - accuracy: 0.6416
Epoch 10/10
782/782 [==============================] - 85s 109ms/step - l

In [164]:
rnn_eval_64_al =  rnn_model_64_al.evaluate(encodings_val_64_padded, y_val)
print(f'RNN Validation Accuracy: {rnn_eval_64_al[1] * 100:.2f}%')

391/391 [==============================] - 14s 34ms/step - loss: 0.6216 - accuracy: 0.6445
RNN Validation Accuracy: 64.45%


In [165]:
gru_model_64_al = create_gru_model_gls((max_length, 64))
gru_model_64_gls.fit(encodings_64_padded, y_train, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 233s 298ms/step - loss: 0.0634 - accuracy: 0.9774
Epoch 2/10
782/782 [==============================] - 230s 294ms/step - loss: 0.0447 - accuracy: 0.9842
Epoch 3/10
782/782 [==============================] - 231s 295ms/step - loss: 0.0344 - accuracy: 0.9886
Epoch 4/10
782/782 [==============================] - 231s 296ms/step - loss: 0.0249 - accuracy: 0.9914
Epoch 5/10
782/782 [==============================] - 231s 295ms/step - loss: 0.0218 - accuracy: 0.9924
Epoch 6/10
782/782 [==============================] - 233s 297ms/step - loss: 0.0281 - accuracy: 0.9902
Epoch 7/10
782/782 [==============================] - 229s 293ms/step - loss: 0.0202 - accuracy: 0.9932
Epoch 8/10
782/782 [==============================] - 230s 294ms/step - loss: 0.0162 - accuracy: 0.9948
Epoch 9/10
782/782 [==============================] - 231s 295ms/step - loss: 0.0125 - accuracy: 0.9958
Epoch 10/10
782/782 [==============================] - 230s 295m

In [167]:
gru_eval_64_al =  gru_model_64_al.evaluate(encodings_val_64_padded, y_val)
print(f'GRU Validation Accuracy: {gru_eval_64_al[1] * 100:.2f}%')

391/391 [==============================] - 30s 77ms/step - loss: 0.6911 - accuracy: 0.5332
GRU Validation Accuracy: 53.32%


In [172]:
# preprocess x_test
stemmer = PorterStemmer()
x_test_preprocessed = []
for i in x_test:
    tokens = word_tokenize(re.sub(r'\d+|[\n\r\t]|<br />', '', i))
    tokens = [word.lower() for word in tokens]
    translator = str.maketrans('','',punctuation)
    tokens = [word.translate(translator) for word in tokens]
    tokens = [word for word in tokens if not word in stop_words]
    tokens = [word for word in tokens if word != ""]
    tokens = [stemmer.stem(word) for word in tokens]
    x_test_preprocessed.append(tokens)
# encoding x_test:
x_test_encoded = [encode_sentence(sentence, top_10000_dict) for sentence in x_test_preprocessed]

x_test_sg_encoded = []
for words in x_test_encoded:
    words_test_sg_encoded = []
    for word in words[:150]:
        word_test_sg_encoded = {}
        for size in vector_sizes:
            word_test_sg_encoded[size] = []
        for size in vector_sizes:
            if word in models[size].wv:
                word_test_sg_encoded[size] = models[size].wv[word]
            else:
                word_test_sg_encoded[size] = models[size].wv['UNK']
        words_test_sg_encoded.append(word_test_sg_encoded)
    x_test_sg_encoded.append(words_test_sg_encoded)
    
encodings_test_32 = separate_encodings(x_test_sg_encoded, 32)
encodings_test_64 = separate_encodings(x_test_sg_encoded, 64)
encodings_test_128 = separate_encodings(x_test_sg_encoded, 128)

encodings_test_32_padded = pad_sequences(encodings_test_32, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_test_64_padded = pad_sequences(encodings_test_64, maxlen=max_length, dtype='float32', padding='post', truncating='post')
encodings_test_128_padded = pad_sequences(encodings_test_128, maxlen=max_length, dtype='float32', padding='post', truncating='post')

In [185]:
y_predict = gru_model_128.predict(encodings_test_128_padded)

391/391 [==============================] - 17s 40ms/step


In [186]:
y_predict

array([[0.72399634],
       [0.11183845],
       [0.02516603],
       ...,
       [0.9941705 ],
       [0.42765546],
       [0.00122641]], dtype=float32)

In [189]:
test_pred_labels = (y_predict >= 0.5).astype(np.float64)

In [188]:
type(y_train[0])

numpy.float64

In [196]:
test_pred_labels

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]])

In [192]:
y_test = np.array(test_pred_labels).flatten()

In [198]:
y_test

array([1., 0., 0., ..., 1., 0., 0.])

In [197]:
len(y_test)

12500

In [199]:
output_file = 'y_test.txt'
with open(output_file, 'w') as file:
    for label in y_test:
        file.write(f"{label}\n")

In [202]:
sequences_test = [text_to_sequence(text, word_to_index) for text in x_test_encoded]

In [203]:
x_test_ft = pad_sequences(sequences_test, maxlen=150 , padding = 'post', value = 10000)

In [210]:
y_predict2 = gru_model_64_ft.predict(x_test_ft)

391/391 [==============================] - 17s 40ms/step


In [211]:
y_predict2

array([[1.8187689e-03],
       [2.4449173e-05],
       [2.7173900e-04],
       ...,
       [9.9972922e-01],
       [3.0726783e-03],
       [7.0648303e-04]], dtype=float32)

In [212]:
test_pred_labels2 = (y_predict2 >= 0.5).astype(np.float64)

In [213]:
test_pred_labels2

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]])

In [214]:
y_test2 = np.array(test_pred_labels2).flatten()

In [215]:
y_test2

array([0., 0., 0., ..., 1., 0., 0.])

In [218]:
output_file = 'y_test2.txt'
with open(output_file, 'w') as file:
    for label in y_test2:
        file.write(f"{label}\n")